In [1]:
import numpy as np
import pandas as pd
import math
import sklearn.preprocessing
import datetime
from TimeBasedCV import TimeBasedCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import make_scorer, r2_score
import statsmodels.api as sm

from sklearn.linear_model import HuberRegressor
pd.set_option('display.max_rows', None)
# more

In [2]:
df = pd.read_csv('factors_2002.csv', parse_dates=['DATE'])
# df = pd.read_csv('data/features_subset.csv', parse_dates=['DATE'])
# df = pd.read_csv('factors_1900.csv', parse_dates=['DATE'])


In [3]:
#Sort observations by date and stock id
df[df.columns[2:]] = df[df.columns[2:]].astype('float32')
df = df.sort_values(by = ['DATE', 'permno'], ascending = True)
df.head()


,permno,DATE,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,macro_dp,macro_ep,macro_bm,macro_ntis,macro_tbl,macro_tms,macro_dfy,macro_svar,Date,macro_mkt-rf
0,10001,2002-01-31,29380.699219,0.024547,0.000603,-0.246853,9.020208,0.029936,-0.032968,6.956505e-03,...,4.274154,3.823618,0.132561,0.011191,0.0165,-0.0027,0.0132,0.002184,200202.0,-1.44
1,10002,2002-01-31,84361.703125,0.149783,0.022435,-0.356072,8.509161,0.058969,0.290509,3.427267e-08,...,4.274154,3.823618,0.132561,0.011191,0.0165,-0.0027,0.0132,0.002184,200202.0,-1.44
2,10012,2002-01-31,65282.878906,2.408649,5.801589,-0.285714,9.949011,0.138741,-0.123827,4.471088e-01,...,4.274154,3.823618,0.132561,0.011191,0.0165,-0.0027,0.0132,0.002184,200202.0,-1.44
3,10019,2002-01-31,10352.500000,1.991428,3.965783,0.214286,7.429473,0.159603,0.039784,4.471088e-01,...,4.274154,3.823618,0.132561,0.011191,0.0165,-0.0027,0.0132,0.002184,200202.0,-1.44
4,10025,2002-01-31,186744.234375,0.138594,0.019208,-0.112840,10.115205,0.070544,0.083402,6.441576e-02,...,4.274154,3.823618,0.132561,0.011191,0.0165,-0.0027,0.0132,0.002184,200202.0,-1.44


In [4]:
# df= df[~np.isnan(df['bm'])]
# df =df[~np.isnan(df['mvel1'])]

In [5]:
df['permno2'] = df['permno'].copy()
df['DATE2'] = df['DATE'].copy()
df = df.set_index(['DATE2','permno2'])

#Make a copy of  the "me" variable (market equity) before rank standartization to use afterwards for value weighting
df['mvel12'] = df['mvel1'].copy()

In [6]:
p=0.3 
df_large= df.groupby('DATE').apply(lambda x: x.nlargest(int(len(x)*p),'mvel1')).reset_index(drop=True)  
df_small = df.groupby('DATE').apply(lambda x: x.nsmallest(int(len(x)*p),'mvel1')).reset_index(drop=True)  


C:\Users\drebi\AppData\Local\Temp\ipykernel_34356\2840026024.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_large= df.groupby('DATE').apply(lambda x: x.nlargest(int(len(x)*p),'mvel1')).reset_index(drop=True)
C:\Users\drebi\AppData\Local\Temp\ipykernel_34356\2840026024.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_small = df.groupby('DATE').apply(lambda x: x.nsmallest(int(len(x)*p),'mvel1'))

In [7]:
#Standardize all independent variables

stdSc = StandardScaler()

features = df.columns[~df.columns.isin(['DATE', 'DATE2', 'mvel1', 'mvel12', 'permno', 'permno2', 'risk_premium'])].tolist()
df[features] = stdSc.fit_transform(df[features].astype(float))


In [8]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   test_period=12,
                   freq='months')

features = df.columns[~df.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()

X = df[features]
y = df[['risk_premium']]

#Empty containers to save results from each window

predictions = []
y_train_list = []
y_val_list = []
y_test_list =[]
dates = []


for train_index, val_index, test_index in tscv.split(X, first_split_date= datetime.date(2007,1,31), second_split_date= datetime.date(2009,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]

    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]

    X_test    = X.loc[test_index].drop('DATE', axis=1)
    y_test  = y.loc[test_index]

    lm = LinearRegression()
   
    lm.fit(X_train, y_train)
    y_pred_train = lm.predict(X_train)
    y_train_list.append(r2_score(y_train, y_pred_train))
    
    y_pred_val = lm.predict(X_val)
    y_val_list.append(r2_score(y_val, y_pred_val))

    y_pred_test = lm.predict(X_test)
    y_test_list.append(r2_score(y_test, y_pred_test))


r2_val_full = np.mean(y_val_list)

Train period: 2002-01-31 - 2007-01-31 ,val period: 2007-01-31 - 2009-01-31 , Test period 2009-01-31 - 2010-01-31 # train records 407907 ,# val records 166149 , # test records 69933
Train period: 2003-01-31 - 2008-01-31 ,val period: 2008-01-31 - 2010-01-31 , Test period 2010-01-31 - 2011-01-31 # train records 403677 ,# val records 154356 , # test records 62636
Train period: 2004-01-31 - 2009-01-31 ,val period: 2009-01-31 - 2011-01-31 , Test period 2011-01-31 - 2012-01-31 # train records 400348 ,# val records 132569 , # test records 67553
Train period: 2005-01-31 - 2010-01-31 ,val period: 2010-01-31 - 2012-01-31 , Test period 2012-01-31 - 2013-01-31 # train records 397015 ,# val records 130189 , # test records 66481
Train period: 2006-01-31 - 2011-01-31 ,val period: 2011-01-31 - 2013-01-31 , Test period 2013-01-31 - 2014-01-31 # train records 379345 ,# val records 134034 , # test records 65816
Train period: 2007-01-31 - 2012-01-31 ,val period: 2012-01-31 - 2014-01-31 , Test period 2014-0

In [9]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   test_period=12,
                   freq='months')


features = df.columns[~df.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()

X = df_large[features]
y = df_large[['risk_premium']]

#Empty containers to save results from each window

predictions = []
y_train_list = []
y_val_list = []
y_test_list =[]
dates = []
dic_r2_all = {}


for train_index, val_index, test_index in tscv.split(X, first_split_date= datetime.date(2007,1,31), second_split_date= datetime.date(2009,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]

    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]

    X_test    = X.loc[test_index].drop('DATE', axis=1)
    y_test  = y.loc[test_index]

    lm = LinearRegression()
   
    lm.fit(X_train, y_train)
    y_pred_train = lm.predict(X_train)
    y_train_list.append(r2_score(y_train, y_pred_train))
    
    y_pred_val = lm.predict(X_val)
    y_val_list.append(r2_score(y_val, y_pred_val))

    y_pred_test = lm.predict(X_test)
    y_test_list.append(r2_score(y_test, y_pred_test))


r2_val_large = np.mean(y_val_list)

Train period: 2002-01-31 - 2007-01-31 ,val period: 2007-01-31 - 2009-01-31 , Test period 2009-01-31 - 2010-01-31 # train records 122347 ,# val records 49836 , # test records 20976
Train period: 2003-01-31 - 2008-01-31 ,val period: 2008-01-31 - 2010-01-31 , Test period 2010-01-31 - 2011-01-31 # train records 121078 ,# val records 46299 , # test records 18787
Train period: 2004-01-31 - 2009-01-31 ,val period: 2009-01-31 - 2011-01-31 , Test period 2011-01-31 - 2012-01-31 # train records 120082 ,# val records 39763 , # test records 20259
Train period: 2005-01-31 - 2010-01-31 ,val period: 2010-01-31 - 2012-01-31 , Test period 2012-01-31 - 2013-01-31 # train records 119083 ,# val records 39046 , # test records 19940
Train period: 2006-01-31 - 2011-01-31 ,val period: 2011-01-31 - 2013-01-31 , Test period 2013-01-31 - 2014-01-31 # train records 113784 ,# val records 40199 , # test records 19738
Train period: 2007-01-31 - 2012-01-31 ,val period: 2012-01-31 - 2014-01-31 , Test period 2014-01-31 

In [10]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   test_period=12,
                   freq='months')


features = df.columns[~df.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()
X = df_small[features]
y = df_small[['risk_premium']]

#Empty containers to save results from each window

predictions_top = []
y_train_list_top =[]
dates_top = []
dic_r2_all_top = {}


for train_index, val_index, test_index in tscv.split(X, first_split_date= datetime.date(2007,1,31), second_split_date= datetime.date(2009,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]

    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]

    X_test    = X.loc[test_index].drop('DATE', axis=1)
    y_test  = y.loc[test_index]

    lm = LinearRegression()
   
    lm.fit(X_train, y_train)
    y_pred_train = lm.predict(X_train)
    y_train_list.append(r2_score(y_train, y_pred_train))
    
    y_pred_val = lm.predict(X_val)
    y_val_list.append(r2_score(y_val, y_pred_val))

    y_pred_test = lm.predict(X_test)
    y_test_list.append(r2_score(y_test, y_pred_test))


r2_val_small = np.mean(y_val_list)

Train period: 2002-01-31 - 2007-01-31 ,val period: 2007-01-31 - 2009-01-31 , Test period 2009-01-31 - 2010-01-31 # train records 122347 ,# val records 49836 , # test records 20976
Train period: 2003-01-31 - 2008-01-31 ,val period: 2008-01-31 - 2010-01-31 , Test period 2010-01-31 - 2011-01-31 # train records 121078 ,# val records 46299 , # test records 18787
Train period: 2004-01-31 - 2009-01-31 ,val period: 2009-01-31 - 2011-01-31 , Test period 2011-01-31 - 2012-01-31 # train records 120082 ,# val records 39763 , # test records 20259
Train period: 2005-01-31 - 2010-01-31 ,val period: 2010-01-31 - 2012-01-31 , Test period 2012-01-31 - 2013-01-31 # train records 119083 ,# val records 39046 , # test records 19940
Train period: 2006-01-31 - 2011-01-31 ,val period: 2011-01-31 - 2013-01-31 , Test period 2013-01-31 - 2014-01-31 # train records 113784 ,# val records 40199 , # test records 19738
Train period: 2007-01-31 - 2012-01-31 ,val period: 2012-01-31 - 2014-01-31 , Test period 2014-01-31 

In [11]:
chart = np.array([[r2_val_full],
                  [r2_val_large ],
                  [r2_val_small]])

r2_lm = pd.DataFrame(chart, columns=['Linear Regression'],
                     index=['Full Sample', 'Large Firms', 'Small Firms'])

r2_lm

,Linear Regression
Full Sample,0.055453
Large Firms,0.152238
Small Firms,0.013234


In [12]:
r2_lm.to_csv(r'r2_linear_model.csv')

In [13]:
features = df.columns[~df.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()
df['year'] = df['DATE'].dt.year

X_train = df[features].loc[(df['year']>=2012) & (df['year']<=2016)]
y_train = df['risk_premium'].loc[(df['year']>=2012) & (df['year']<=2016)]

X_val = df[features].loc[(df['year']>=2012) & (df['year']<=2016)]
y_val = df['risk_premium'].loc[(df['year']>=2012) & (df['year']<=2016)]

In [14]:
X_train.drop(columns=['DATE'], inplace=True)
y_train.drop(columns=['DATE'], inplace=True)

X_val.drop(columns=['DATE'], inplace=True)
y_val.drop(columns=['DATE'], inplace=True)

In [15]:

lin_model = LinearRegression()
lin_model.fit(X_train, y_train)
y_pred_train = lin_model.predict(X_train) 

y_pred_val = lin_model.predict(X_val)

r2_score(y_val, y_pred_val)

0.08439209665810532

In [8]:

for j in features:
    globals()['df_' + str(j)] =  df.copy()
    globals()['df_' + str(j)][str(j)] = 0

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\drebi\miniconda3\envs\statclass\Lib\site-packages\IPython\core\interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\drebi\AppData\Local\Temp\ipykernel_34356\2700331551.py", line 2, in <module>
    globals()['df_' + str(j)] =  df.copy()
                                 ^^^^^^^^^
  File "c:\Users\drebi\miniconda3\envs\statclass\Lib\site-packages\pandas\core\generic.py", line 6811, in copy
    data = self._mgr.copy(deep=deep)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\drebi\miniconda3\envs\statclass\Lib\site-packages\pandas\core\internals\managers.py", line 604, in copy
    res._consolidate_inplace()
  File "c:\Users\drebi\miniconda3\envs\statclass\Lib\site-packages\pandas\core\internals\managers.py", line 1788, in _consolidate_inplace
    self.blocks = _consolidate(self.blocks)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\drebi\miniconda3\envs\stat

In [21]:
dic = {}

    
for j in features:
    df_var = globals()['df_' + str(j)]
    
    X_train = df_var[features].loc[(df_var["year"]>=2012) & (df_var["year"]<=2016)]
    y_train = df_var["ret_exc_lead1m"].loc[(df_var["year"]>=2012) & (df_var["year"]<=2016)]

    X_val = df_var[features].loc[(df_var["year"]>=2017) & (df_var["year"]<=2018)]
    y_val = df_var["ret_exc_lead1m"].loc[(df_var["year"]>=2017) & (df_var["year"]<=2018)]
    
    reg_huber = HuberRegressor(max_iter=1000)
    reg_huber.fit(np.concatenate((X_train, X_val)), np.concatenate((y_train, y_val)))
    preds = reg_huber.predict(np.concatenate((X_train, X_val))) 

    R2OOS_var = 1-sum(pow(np.concatenate((y_train, y_val))-preds,2))/sum(pow(np.concatenate((y_train, y_val)),2))
    dic['R2OOS_' + str(j)] = R2OOS_var

KeyError: 'df_DATE'